# Project: Advanced Lane Lines

In this Notebook, I am going to describe the pipeline used to detect lane Line and its curvature in a more sofisticated way than the first lane detection project.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import copy
import imageio
imageio.plugins.ffmpeg.download()
from IPython.display import HTML
from moviepy.editor import VideoFileClip


## Camera Clibration
### Finding corners

In [2]:
def retrieve_distortion_points(image_shape=(1280, 720)):
    nx = 9
    ny = 6
    c=1
    
    objpoints=[]
    imgpoints=[]
    objp = np.zeros((nx*ny,3), np.float32)
    for i in range(ny):
        for j in range(nx):
            objp[j+nx*i,:]=(j, i, 0)

    # Go through images in folder collect data for calibration
    for fname in glob.glob('camera_cal/*.jpg'):
        img = cv2.imread(fname)
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        # If found, draw corners
        print('Image: ', c, " - Retrieved corners: ", ret )
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
        c=c+1
            
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image_shape, None, None)
    return mtx, dist



## Line Detection (using Color filtering and Sobel)

In [3]:
def color_filt(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    imshape=img.shape
    # Create lower and upper boundaries for the range of the hls-values of the driving-surface in front of the car
    pavement_color_low=np.percentile(hls[int(imshape[0]*0.70):int(imshape[0]*0.95),int(imshape[1]*0.45):int(imshape[1]*0.55),:],25, axis=(0,1))
    pavement_color_high=np.percentile(hls[int(imshape[0]*0.70):int(imshape[0]*0.95),int(imshape[1]*0.45):int(imshape[1]*0.55),:],75, axis=(0,1))
 
    #Hue threshold. Only colors in that range are used for lane detection in the saturation channel
    h_thresh_min = 10
    h_thresh_max = 60
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh_min) & (h_channel <= h_thresh_max)] = 1
    
    #Lightness threshold. It shows everay area darker than the driving surface. this area is later deleted from the Sobel edge detection
    l_neg_thresh_min = 0
    l_neg_thresh_max = np.minimum(pavement_color_low[1]+20,70)
    l_neg_binary = np.zeros_like(l_channel)
    l_neg_binary[(l_channel >= l_neg_thresh_min) & (l_channel <= l_neg_thresh_max)] = 1
    #It is dilated horizontally to also capture the horiz. edges of the unwanted areas
    kernel = np.ones((1,14),np.uint8)
    l_neg_binary=cv2.dilate(l_neg_binary,kernel,iterations = 1)


    s_thresh_min = pavement_color_high[2]+40
    s_thresh_max = 255
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1



    comb_binary_c=copy.copy(s_binary)
    comb_binary_c[h_binary==0]=0
    
    #Create new Image with equalized colors of the driving-surface
    lnew=np.copy(l_channel)
    lnew[l_channel<pavement_color_high[1]]=pavement_color_high[1]
    new_hls=np.dstack((h_channel,lnew,s_channel))
    
    img_new=cv2.cvtColor(new_hls, cv2.COLOR_HLS2RGB)
    
    return comb_binary_c, img_new, l_neg_binary

In [4]:
# in order to minimize redundancies in later calculations, I created one function for all sobel operations
def sobel_thresh(gray, sobel_kernel, orient, orient_thresh, dir_thresh, mag_thresh):
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    #Gradient magnitude in one direction
    orient_binary=np.zeros_like(gray)
    if orient=='x':
        abs_sobel=np.absolute(sobelx)
    if orient=='y':
        abs_sobel=np.absolute(sobely)
        
    abs_sobel=np.uint8(255*abs_sobel/(np.max(abs_sobel)))
    flags_o=(abs_sobel >= orient_thresh[0])&(abs_sobel <=orient_thresh[1])
    orient_binary[flags_o]=1
    
    #Directional gradient filter
    dir_binary=np.zeros_like(gray)
    sobelxy=np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    flags_d=(sobelxy >= dir_thresh[0])&(sobelxy <=dir_thresh[1])
    dir_binary[flags_d]=1
    
    #Total magnitude gradient filter
    mag_binary=np.zeros_like(gray)
    sobelm=np.sqrt(sobelx*sobelx+sobely*sobely)
    sobelm=np.uint(255*sobelm/np.max(sobelm))
    flags_m=(sobelm >= mag_thresh[0])&(sobelm <=mag_thresh[1])
    mag_binary[flags_m]=1
        
    return orient_binary, dir_binary, mag_binary

In [5]:
def filter_for_lane_lines(img):

    #First apply color_filter
    comb_binary_c, img_new, l_neg_binary=color_filt(img)
    
    #Use modified Image (equilized lightness) for gray image for edge detection
    gray=cv2.cvtColor(img_new, cv2.COLOR_RGB2GRAY)

    ### Parameters for Sobel
    ksize = 7
    # Gradient magnitude in one direction --> Later illustrated as red
    orient='x'
    orient_thresh=(30, 100)
    # Directional gradient filter --> Later illustrated as green
    dir_thresh=(0.7, 1.3)
    #Total magnitude gradient filter --> Later illustrated as blue
    mag_thresh=(50, 255)

    # Apply each of the thresholding functions
    orient_binary, dir_binary, mag_binary = sobel_thresh(gray, sobel_kernel=ksize, orient=orient, orient_thresh=orient_thresh, dir_thresh=dir_thresh, mag_thresh=mag_thresh)
    
    #Take all edges that two off the three sobel_filters dected
    comb_binary = np.zeros_like(gray)
    twos=np.ones_like(gray)*2
    comb_binary[(orient_binary+dir_binary+mag_binary)>=twos] = 1
    #subtract dark areas from detected edges
    #comb_binary[(l_neg_binary)==1]= 0
    #add detected lanes from color filter
    comb_binary[(comb_binary_c == 1)]= 1
    return comb_binary

### Warp image and histogram

In [6]:
def warp_image_param(image_width,osize,horizon):
    #Parameters of warping Trapezoid--> Square
    
    #Lower left corner
    low_y=670
    low_x=255
    
    #Upper left corner
    if horizon==0: #long wide highway
        top_y=440
        top_x=607
        o_sidemargin=osize/3 #margin if something is found left and right of the center lane (--> Curve)
        
        
    if horizon==1: #hilly highway
        top_y=460
        top_x=580
        o_sidemargin=osize/4.5 #margin if something is found left and right of the center lane (--> Curve)
        
    if horizon==2: #countryside_road
        top_y=500
        top_x=510
        o_sidemargin=osize/3 #margin if something is found left and right of the center lane (--> Curve)
    
    
    
    src=np.float32([[low_x,low_y],
                    [top_x,top_y],
                    [image_width-top_x,top_y],
                    [image_width-low_x,low_y]])
    dst=np.float32([[o_sidemargin,osize],
                    [o_sidemargin,0],
                    [osize-o_sidemargin,0],
                    [osize-o_sidemargin,osize]])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)

    return M, Minv


### Line Detection using sliding windows
Used initially or if the previous detection seemed to have failed

In [7]:
def initial_line_detector(binary_warped):
    # Take a histogram of the bottom Half of the image
    start_of_hist=int(binary_warped.shape[0]*0.5)
    histogram = np.sum(binary_warped[start_of_hist:,:], axis=0)
    

    # Peak of the left and right >halves of the histogram
    midpoint = np.int(histogram.shape[0]//2)
    lane_mark_width=10
    
    # introduce a margin arround the border of the pixel where no stating point is searched
    leftx_base = np.argmax(histogram[lane_mark_width:midpoint])+lane_mark_width 
    rightx_base = np.argmax(histogram[midpoint:-lane_mark_width]) + midpoint


    # Choose the number of sliding windows
    nwindows = 15
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 10
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    #Create Penalty counter, for when the lane (and thus one window boundary) leaves the projected image
    out_r=0
    out_l=0

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
       
        
        if (win_xleft_low<-margin/2) or (win_xleft_high>binary_warped.shape[1]+margin/2):
            out_l=out_l+2
        elif out_l>0:
            out_l=out_l-1
            
        if (win_xright_low<-margin/2) or (win_xright_high>binary_warped.shape[1]+margin/2):
            out_r=out_r+2
        elif out_r>0:
            out_r=out_r-1
            
            
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # If panelty counter is low and less the 90% of the Rectangle is filled with ONES --> Append these indices to the lists
        if out_l<4 and (len(good_left_inds) < window_height*margin*1.8):
            left_lane_inds.append(good_left_inds)
        if out_r<4 and (len(good_right_inds) < window_height*margin*1.8):
            right_lane_inds.append(good_right_inds)
            
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.median(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.median(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # If pixels are found
    # Extract left and right line pixel positions
    # Fit a second order polynomial to each
    
    # since my line detection is quite aggressive here I avoid an error by givin back(0,0,0) if less than 3 pixels are found  
    if len(nonzerox[left_lane_inds])>3:
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds]
        left_fit = np.polyfit(osize-lefty, leftx, 2)
    else:
        left_fit=np.array([0,0,0], dtype='float')
    
    if len(nonzerox[right_lane_inds])>3:
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        right_fit = np.polyfit(osize-righty, rightx, 2)
    else:
        right_fit=np.array([0,0,0], dtype='float')

    return left_fit, right_fit

### Line Detection using area around previous fit
Used recurrently if the previous detection seemed to yielded an acceptable result

In [8]:
def recurrent_line_detector(binary_warped, left_fit, right_fit):
    nonzero = binary_warped.nonzero()
    nonzeroy = osize-np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    margin = 100 #margin arround the current fit in which a new fit is searched
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  
    
    
    if len(nonzerox[left_lane_inds])>3:
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds]
        left_fit = np.polyfit(lefty, leftx, 2)
    else:
        left_fit=np.array([0,0,0], dtype='float')
    
    if len(nonzerox[right_lane_inds])>3:
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        right_fit = np.polyfit(righty, rightx, 2)
    else:
        right_fit=np.array([0,0,0], dtype='float')

    
    return  left_fit, right_fit

### Curvature and offset detection
parameters are adjusted, depending on the driving mode

In [9]:
def find_curvature_and_offset(left_fitx, right_fitx, horizon):
    y_eval = np.max(ploty)
    if horizon==0:
        ym_per_pix = 26/y_eval # pixel per meter in y dimension
        xm_per_pix = 3.7/y_eval*0.38 # meters per pixel in x dimension
        
    if horizon==1:
        ym_per_pix = 30/y_eval # pixel per meter in y dimension
        xm_per_pix = 3.7/y_eval*0.65 # meters per pixel in x dimension
        
    if horizon==2:
        ym_per_pix = 40/y_eval # pixel per meter in y dimension
        xm_per_pix = 3.7/y_eval*0.38 # meters per pixel in x dimension
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)

    
    # Calculate the new radii of curvature 
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    
    offset=(left_fitx[0]+ right_fitx[0]-osize)/2.0*xm_per_pix
    
    return left_curverad, right_curverad, offset

### Create the final Frame
Including warping the detected line into the correct perspective,
Plotting the area of the lane, writing current curvature and postion in lane

In [10]:
def create_lane_marked(img,binary_warped,left_fitx, right_fitx, left_curverad, right_curverad, offset):
    warp_zero = np.zeros((osize,osize)).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, osize-ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, osize-ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)#[:,int(img.shape[1]/2):int(img.shape[1]*3/2),:]
    
    # Create a mean curvature and bring it into a nice form
    mean_curve=(left_curverad+right_curverad)//2
    if mean_curve>9800:
        mean_curve_s='inf'
    else:
        mean_curve_s=str(int(round(mean_curve, -2)))
    
    #Name offset side
    if offset>0:
        lor='left'
    else:
        lor='right'

    font=cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result,'Mean Curvature : %s m' %mean_curve_s, (10,650), font, 1, (0,0,0),3)
    cv2.putText(result,'Offset: %10.4f m %s of center' %(np.abs(offset), lor), (10,680), font, 1, (0,0,0),3)
    return result

### Line class

In [11]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = True  
        # x values of the last n fits of the line
        self.xfitted = None
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = np.array([0,0,0], dtype='float')
        #polynomial coefficients for the most recent fit
        self.current_fit = np.array([0,0,0], dtype='float')
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float')

---

## Main Pipeline
- Calculations that only have to be done once but whose output is used globally
- Initialization of other global variables

In [12]:
#Camera Clibration/Distortion
mtx, dist=retrieve_distortion_points()

#Size of quadratic Binary Image
osize=1024

#Imformation about the source-movie
Framerate=30
image_width=1280

#Horizon of the warped image: curvier roads have a closer horizon
# 0: wide highway
# 1: narrow, hilly highway
# 2: countryside-road
horizon=0 

#Create Parameters for warping (both directions)
M,Minv=warp_image_param(image_width,osize,horizon)

#Number of frames over which is smoothed
av=5

R_line=Line() #Creates Right Line Class
L_line=Line() #Creates Left Line Class
ploty = np.linspace(osize-1,0, osize)
R_line.xfitted=np.zeros((av,osize))#*osize*0.25
L_line.xfitted=np.zeros((av,osize))#*osize*0.75
L_line.radius_of_curvature=0
R_line.radius_of_curvature=0
offset=0

Image:  1  - Retrieved corners:  False
Image:  2  - Retrieved corners:  True
Image:  3  - Retrieved corners:  True
Image:  4  - Retrieved corners:  True
Image:  5  - Retrieved corners:  True
Image:  6  - Retrieved corners:  True
Image:  7  - Retrieved corners:  True
Image:  8  - Retrieved corners:  True
Image:  9  - Retrieved corners:  True
Image:  10  - Retrieved corners:  True
Image:  11  - Retrieved corners:  True
Image:  12  - Retrieved corners:  True
Image:  13  - Retrieved corners:  True
Image:  14  - Retrieved corners:  True
Image:  15  - Retrieved corners:  False
Image:  16  - Retrieved corners:  False
Image:  17  - Retrieved corners:  True
Image:  18  - Retrieved corners:  True
Image:  19  - Retrieved corners:  True
Image:  20  - Retrieved corners:  True


### The process that is applied to every frame

In [13]:
def process_frame(get_frame,t):
    #get frame from time
    f=int(t*Framerate)
    
    img=get_frame(t)
 
    #undistort
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
    #Create Mask/Binary
    binary=filter_for_lane_lines(undistorted)
    #Warp Binary
    binary_warped = cv2.warpPerspective(binary, M, (osize,osize), flags=cv2.INTER_LINEAR)
    
        
    #Line Marking Detection
    
    if f<1 or R_line.detected==False or L_line.detected==False:
        #using sliding windows
        L_line.current_fit, R_line.current_fit=initial_line_detector(binary_warped)
    else:
        #using an area around a predefined, known lane marking from the previous frame
        L_line.current_fit, R_line.current_fit=recurrent_line_detector(binary_warped, L_line.best_fit, R_line.best_fit)
   
    
    # Reset Line Detection
    L_line.detected=True
    R_line.detected=True
    
    
    ########### Rating of current fit#############
    
    #Line parameter difference between frames for each line
    if f>0:
        L_line.diffs=L_line.current_fit-L_line.best_fit
        R_line.diffs=R_line.current_fit-R_line.best_fit
        
        # Newly detected line compared to line on other side, considering divergence
        avg_curve=(L_line.best_fit[0:1]+R_line.best_fit[0:1])/2.0
        if (L_line.current_fit[0]>0 and avg_curve[0]<0) or\
            (L_line.current_fit[0]<0 and avg_curve[0]>0):
                L_line.current_fit=L_line.best_fit+R_line.diffs
                L_line.detected=False
                #annot[0]='d'
        if (R_line.current_fit[0]>0 and avg_curve[0]<0) or\
            (R_line.current_fit[0]<0 and avg_curve[0]>0):            
                R_line.current_fit=R_line.best_fit+L_line.diffs
                R_line.detected=False
                #annot[4]='d'
                
        # Newly detected line starting point verification (just before car)
        if L_line.current_fit[2]>osize/2 or L_line.current_fit[2]<-100:
                L_line.detected=False
                #annot[1]='S'
        if R_line.current_fit[2]>osize+100 or R_line.current_fit[2]<osize/2:
                L_line.detected=False
                #annot[5]='S'
                
        #Newly polynom of detected line differs from averaged previous lines
        if np.any(np.abs(L_line.diffs)>(1e-3, 2e-1, 50)):
                #annot[2]='D'
                L_line.detected=False
                L_line.current_fit=L_line.best_fit
        if np.any(np.abs(R_line.diffs)>(1e-3, 2e-1, 50)):
                R_line.detected=False
                R_line.current_fit=R_line.best_fit
                #annot[6]='D'
    
    ########### Current fit is added to exponential average#############
    
    if f==0:
        #Initialization in first frame
        L_line.best_fit=L_line.current_fit
        R_line.best_fit=R_line.current_fit
        
    if  L_line.detected==True:
        #If line is detected add it to the exponential average
        L_line.best_fit=1.0/av*L_line.current_fit+(1-1.0/av)*L_line.best_fit
    else:
        #If line is not detected still add it to the exponential average, but with a quarter of the weight
        L_line.best_fit=1.0/(av*4)*L_line.current_fit+(1-1.0/(av*4))*L_line.best_fit
                
    if R_line.detected==True:
        #If line is detected add it to the exponential average
        R_line.best_fit=1.0/av*R_line.current_fit+(1-1.0/av)*R_line.best_fit
    else:
        #If line is not detected still add it to the exponential average, but with a quarter of the weight
        L_line.best_fit=1.0/(av*4)*L_line.current_fit+(1-1.0/(av*4))*L_line.best_fit
        
        
    ########## Using current best fit to create the line ######
    
    left_fitx = L_line.current_fit[0]*ploty**2 + L_line.current_fit[1]*ploty + L_line.current_fit[2]
    right_fitx = R_line.current_fit[0]*ploty**2 + R_line.current_fit[1]*ploty + R_line.current_fit[2]
    
    
    #add current line (if there was a line detected in the recent frames) to an array of 5 lines
    if f<av:
        L_line.xfitted[f,:]=left_fitx
    elif not(np.any(L_line.best_fit==(0,0,0))):
        L_line.xfitted=np.roll(L_line.xfitted, 1,axis=0)
        L_line.xfitted[0,:]=left_fitx
        
    if f<av:
        R_line.xfitted[f,:]=right_fitx
    elif not(np.any(R_line.best_fit==(0,0,0))):
        R_line.xfitted=np.roll(R_line.xfitted, 1,axis=0)
        R_line.xfitted[0,:]=right_fitx

    
    #find curvature of best current fit (median-values of last n lines that where found; median dismisses 'hiccups')
    l_curve, r_curve , offset=find_curvature_and_offset(np.median(L_line.xfitted, axis=0), np.median(R_line.xfitted, axis=0),horizon)
    
    
    # even out the curvature over the last secound (roughly) by using exponential average, takes care that Radius will never exceed 10000    
    L_line.radius_of_curvature=1.0/(av*5)*np.minimum(l_curve,10000)+(1-1.0/(av*5))*L_line.radius_of_curvature
    R_line.radius_of_curvature=1.0/(av*5)*np.minimum(r_curve,10000)+(1-1.0/(av*5))*R_line.radius_of_curvature
       
    #create result
    result=create_lane_marked(img,binary_warped, np.median(L_line.xfitted, axis=0), np.median(R_line.xfitted, axis=0), L_line.radius_of_curvature, R_line.radius_of_curvature, offset)
    
    
    return result
    
    
    

In [14]:
video_output = 'output_images/project_video.mp4'


clip1 = VideoFileClip("project_video.mp4")


white_clip = clip1.fl(process_frame)#.subclip(0,10)
%time white_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video output_images/project_video.mp4
[MoviePy] Writing video output_images/project_video.mp4


100%|█████████▉| 1260/1261 [05:26<00:00,  3.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video.mp4 

Wall time: 5min 27s
